# imports



In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset, concatenate_datasets
import random
import pandas as pd
import re
import pickle
import matplotlib.pyplot as plt

from  bs4 import BeautifulSoup
import requests
import time
import os 

# parse en ru dict

In [ ]:
url_base = "http://www.7english.ru/dictionary.php"
urls = ['?id=500&letter=all',  '?id=1000&letter=all', '?id=2000&letter=all',  '?id=3500&letter=all', '?id=6000&letter=all', '?id=2&letter=all', '?id=3&letter=all']

In [ ]:

def parse_words(url):
    rs = requests.get(url)
    root = BeautifulSoup(rs.content, 'html.parser')

    en_list, ru_list = [], []

    for tr in root.select('tr[onmouseover]'):
        td_list = [td.text.strip() for td in tr.select('td')]

        # Количество ячеек в таблице со словами -- 9
        if len(td_list) != 9 or not td_list[1] or not td_list[5]:
            continue

        en = td_list[1]

        # Русские слова могут быть перечислены через запятую 'ты, вы',
        # а нам достаточно одного слова
        # 'ты, вы' -> 'ты'
        ru = td_list[5].split(', ')[0]

        en_list.append(en)
        ru_list.append(ru)
    return pd.DataFrame({'en': en_list, 'ru': ru_list})



In [ ]:
res = pd.DataFrame()
for url in urls:
  df = parse_words(url_base + url)
  print(df.size / 2 )
  res = res.append(df)

In [ ]:
res.head(5)

In [ ]:
res = res.drop_duplicates()[1:]

In [ ]:
res.sort_values('en').to_csv('drive/MyDrive/ML/enru_dict.csv', index=False)

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def parse_sent(url):
    rs = requests.get(url)
    root = BeautifulSoup(rs.content, 'html.parser')

    themes, en_list, ru_list =[], [], []

    for tr in root.select('tr[onmouseover]'):
        td_list = [(td.text.strip(), td.select('a')) for td in tr.select('td')]

        # Количество ячеек в таблице со словами -- 9
        if len(td_list) != 9 or not td_list[1][0] or not td_list[5][0] or not td_list[5][1]:
            continue

        theme = td_list[1][0]

        # Русские слова могут быть перечислены через запятую 'ты, вы',
        # а нам достаточно одного слова
        # 'ты, вы' -> 'ты'
        en = td_list[5][0].replace('[перевод]', "")
        ru = td_list[5][1][0].get('title', 'No title')

        themes.append(theme)
        en_list.append(en)
        ru_list.append(ru)
    return pd.DataFrame({'theme': themes,'en': en_list, 'ru': ru_list})


In [ ]:
parse_sent('http://www.7english.ru/dictionary.php?id=5&letter=33').head(5)

# cambridge en ru dict

In [ ]:
headers =  {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36"}

alphabet=['a', 'b']
en_dict = {}
url = "https://dictionary.cambridge.org"
for litera in alphabet:
  
  resp = requests.get(f"https://dictionary.cambridge.org/browse/english-russian/{litera}", headers=headers)
  root = BeautifulSoup(resp.content)
  href_list = [h.get('href') for h in root.find_all("a", class_="dil tcbd")]

  for href in href_list:
    resp = requests.get(href, headers=headers)
    root = BeautifulSoup(resp.content)
    href_sub_list = [h.get('href') for h in root.find_all("a", class_="tc-bd")][1:]

    for sub_href in href_sub_list:
      resp = requests.get(url + sub_href, headers=headers)
      root = BeautifulSoup(resp.content)
      en = root.find("div", class_="h2 dhw dpos-h_hw lpt-5 lpb-10")
      ru = root.find("span", class_="trans dtrans dtrans-se ")
      if en and ru:
        en_dict[en.text.strip()] = [ru.text.strip()]
        examples = root.find_all("div", class_="lbb lb-cm lpt-10")
        for example in examples:
          text = example.find("span", class_="deg").text.strip()
          en_dict[en.text.strip()].append(text)


# filter HF datset and group by words

In [ ]:

translate_dataset = load_dataset('opus100', 'en-ru', )

enru_dict = pd.read_csv('drive/MyDrive/ML/enru_dict.csv')
en_dict = {item.en: [{'en': item.en, 'ru': item.ru}] for i, item in enru_dict.iterrows()}
en_dict['<heap>'] = []

In [ ]:

ru_regex = re.compile(r"[а-яА-ЯёЁ]")
en_regex = re.compile(r"[a-zA-Z]")

for example in translate_dataset['train']['translation']:

  no_russian_words = len(ru_regex.findall(example['ru'])) < 0.5 * len(example['ru'])
  no_en_words = len(en_regex.findall(example['en'])) < 0.5 * len(example['en'])
  if no_russian_words or no_en_words or abs(1 - len(example['ru'])/len(example['en'])) > 0.4 :
    continue

  last_matched_token = None
  is_appened = False
  for token in example['en'].split():
    if token in en_dict:
      last_matched_token = token
      if len(en_dict[token]) < 150:
        en_dict[token].append(example)
        is_appened = True
        break
  if not is_appened:      
    if last_matched_token:
      en_dict[last_matched_token].append(example)
        
    else:
      en_dict['<heap>'].append(example)

en_dict = dict(sorted(en_dict.items(), key=lambda x: len(x[1]), reverse=True))

with open('drive/MyDrive/ML/hf_dataset.pickle', 'wb') as f:
  pickle.dump(en_dict, f)

In [ ]:
len([k for k, v in en_dict.items() if len(v) < 20])

# parse examples from internet


In [ ]:
headers =  {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36"}
enru_dict = pd.read_csv('drive/MyDrive/ML/enru_dict.csv')

if os.path.exists('drive/MyDrive/ML/parsed_dataset.pickle'):
  with open('drive/MyDrive/ML/parsed_dataset.pickle', 'rb') as f:
    examples_dict = pickle.load(f)
else:
  examples_dict = {}

In [ ]:

for i, item in enru_dict.iterrows():

  if not examples_dict.get(item.en):
    if (i + 1) % 100 == 0:
      time.sleep(30)

    examples_dict[item.en] = []
    resp = requests.get(f"https://context.reverso.net/translation/english-russian/{item.en}", headers=headers)
    root = BeautifulSoup(resp.content)

    for en, ru in zip(root.find_all('div', class_="src ltr", limit=100), root.find_all('div', class_="trg ltr", limit=100)):
      
      examples_dict[item.en].append({'en': en.text.strip(), 'ru': ru.text.strip()})
  




In [ ]:
list(examples_dict.items())[-1]

In [ ]:
import pickle
with open('drive/MyDrive/ML/parsed_dataset.pickle', 'wb') as f:
  pickle.dump(examples_dict, f)

# cat datasets

In [ ]:
with open('drive/MyDrive/ML/hf_dataset.pickle', 'rb') as f:
    hf_dataset = pickle.load(f)

In [ ]:
with open('drive/MyDrive/ML/parsed_dataset.pickle', 'rb') as f:
    parsed_dataset = pickle.load(f)

In [ ]:
for key in hf_dataset.keys():
  hf_dataset[key] += parsed_dataset.get(key,[])

In [ ]:
len([k for k, v in hf_dataset.items() if len(v) < 100])

In [ ]:
with open('drive/MyDrive/ML/translation_dataset.pickle', 'wb') as f:
    pickle.dump(hf_dataset, f)